# Comparing MLP vs. KAN on Classification problem with a bit more input dimensions (Fashion MNIST)

Original KAN example: https://github.com/KindXiaoming/pykan/blob/master/tutorials/Example_3_classfication.ipynb \
EfficientKAN by Blealtan: https://github.com/Blealtan/efficient-kan \
MLP in pytorch Referenced from: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html \
ConvolutionalKAN by AntonioTepsich: https://github.com/AntonioTepsich/Convolutional-KANs

Test with Fashion MNIST dataset: https://github.com/zalandoresearch/fashion-mnist


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

import pickle
import struct
from array import array

from tqdm import tqdm
import time

import gc
import os
from os.path import join
import sys
import math

In [2]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'efficient-kan-master', 'src')))

from efficient_kan import KAN 

In [3]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'Convolutional-KANs-master')))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'Convolutional-KANs-master', 'kan_convolutional')))

from kan_convolutional.KANConv import KAN_Convolutional_Layer
from kan_convolutional.KANLinear import KANLinear

In [4]:
print('Has CUDA:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('device name:', torch.cuda.get_device_name(0))

Has CUDA: True
device name: NVIDIA GeForce RTX 3060


### Prepare the dataset

##### Read from file

In [5]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), 'fashion-mnist-master', 'utils')))

In [6]:
# load with utility code
import mnist_reader
x_tr, y_tr = mnist_reader.load_mnist(join('fashion-mnist', 'data', 'fashion'), kind='train')
x_te, y_te = mnist_reader.load_mnist(join('fashion-mnist', 'data', 'fashion'), kind='t10k')

In [7]:
x_tr = x_tr.reshape((x_tr.shape[0], 28, 28))
x_te = x_te.reshape((x_te.shape[0], 28, 28))

##### Data processing

In [8]:
# normalie the dataset values to [0.0, 1.0]
vmax = np.amax(x_tr)
x_tr = x_tr / vmax
x_te = x_te / vmax

In [9]:
# flatten
x_tr_flattened = x_tr.reshape((x_tr.shape[0], x_tr.shape[1] * x_tr.shape[2]))
x_te_flattened = x_te.reshape((x_te.shape[0], x_te.shape[1] * x_te.shape[2]))

# conv color channel
x_tr_channeled = x_tr.reshape((x_tr.shape[0], 1, x_tr.shape[1], x_tr.shape[2]))
x_te_channeled = x_te.reshape((x_te.shape[0], 1, x_te.shape[1], x_te.shape[2]))

In [10]:
x_train_flattened = torch.tensor(np.array(x_tr_flattened))
x_train_channeled = torch.tensor(np.array(x_tr_channeled))
y_train = torch.tensor(np.array(y_tr))

x_test_flattened  = torch.tensor(np.array(x_te_flattened))
x_test_channeled  = torch.tensor(np.array(x_te_channeled))
y_test  = torch.tensor(np.array(y_te))

In [11]:
x_train_flattened.shape

torch.Size([60000, 784])

In [12]:
x_train_channeled.shape

torch.Size([60000, 1, 28, 28])

In [13]:
# chop off a ratio of the train sets
ratio_keep = 1.0

num_train = x_tr.shape[0]
num_test = x_te.shape[0]

num_train_1 = int(num_train * ratio_keep)
num_test_1  = int(num_test * ratio_keep)

x_train_flattened = x_train_flattened[:num_train_1, :]
x_train_channeled = x_train_channeled[:num_train_1, :]
y_train = y_train[:num_train_1]

x_test_flattened = x_test_flattened[:num_test_1, :]
x_test_channeled = x_test_channeled[:num_test_1, :]
y_test = y_test[:num_test_1]

##### Create a troch dataset object

In [14]:
from torch.utils.data import TensorDataset, DataLoader

In [15]:
batch_size = 256

In [16]:
# flattened for fully connected
train_set_flattened = TensorDataset(x_train_flattened.to(dtype=torch.float32), y_train)
train_loader_flattened = DataLoader(train_set_flattened, batch_size=batch_size, shuffle=True)

test_set_flattened = TensorDataset(x_test_flattened.to(dtype=torch.float32), y_test)
test_loader_flattened = DataLoader(test_set_flattened, batch_size=batch_size, shuffle=False)

# channeled for convolution
train_set_channeled = TensorDataset(x_train_channeled.to(dtype=torch.float32), y_train)
train_loader_channeled = DataLoader(train_set_channeled, batch_size=batch_size, shuffle=True)

test_set_channeled = TensorDataset(x_test_channeled.to(dtype=torch.float32), y_test)
test_loader_channeled = DataLoader(test_set_channeled, batch_size=batch_size, shuffle=False)

In [17]:
dataset_flattened = {'train': train_loader_flattened, 'test': test_loader_flattened}
dataset_channeled = {'train': train_loader_channeled, 'test': test_loader_channeled}

### Code for classical MLP

In [18]:
class NeuralNetwork(nn.Module):
    '''
    width is the dimension of each layer, like [784, 20, 20, 10]
    '''
    def __init__(self, width):
        super().__init__()
        self.flatten = nn.Flatten()
        stack = []
        l = len(width)
        for i in range(l-2):
            stack.append(nn.Linear(width[i], width[i+1]))
            stack.append(nn.ReLU())
        stack.append(nn.Linear(width[l-2], width[l-1]))
        
        self.linear_relu_stack = nn.Sequential(*stack)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

### Code for ConvolutionKAN
copied from: https://github.com/AntonioTepsich/Convolutional-KANs/tree/master/architectures_28x28

In [19]:
class ConvKAN_2conv(nn.Module):
    def __init__(self,device: str = 'cuda'):
        super().__init__()
        self.conv1 = KAN_Convolutional_Layer(
            n_convs = 5,
            kernel_size= (3,3),
            device = device
        )

        self.conv2 = KAN_Convolutional_Layer(
            n_convs = 5,
            kernel_size = (3,3),
            device = device
        )

        self.pool1 = nn.MaxPool2d(
            kernel_size=(2, 2)
        )
        
        self.flat = nn.Flatten() 
        
        self.linear1 = nn.Linear(625, 64)
        self.linear2 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.conv1(x)

        x = self.pool1(x)

        x = self.conv2(x)
        x = self.pool1(x)
        x = self.flat(x)
        x = self.linear1(x)
        x = self.linear2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [20]:
class ConvKAN_1conv(nn.Module):
    def __init__(self,device: str = 'cuda'):
        super().__init__()
        self.conv1 = KAN_Convolutional_Layer(
            n_convs = 5,
            kernel_size= (3,3),
            device = device
        )

        self.pool1 = nn.MaxPool2d(
            kernel_size=(2, 2)
        )
        
        self.flat = nn.Flatten() 
        
        self.linear1 = nn.Linear(845, 64)
        self.linear2 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.flat(x)
        x = self.linear1(x)
        x = self.linear2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [21]:
class KKAN_Convolutional_Network(nn.Module):
    def __init__(self,device: str = 'cuda'):
        super().__init__()
        self.conv1 = KAN_Convolutional_Layer(
            n_convs = 5,
            kernel_size= (3,3),
            device = device
        )

        self.conv2 = KAN_Convolutional_Layer(
            n_convs = 5,
            kernel_size = (3,3),
            device = device
        )

        self.pool1 = nn.MaxPool2d(
            kernel_size=(2, 2)
        )
        
        self.flat = nn.Flatten() 

        self.kan1 = KANLinear(
            625,
            10,
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0,1],
        )


    def forward(self, x):
        x = self.conv1(x)

        x = self.pool1(x)

        x = self.conv2(x)
        x = self.pool1(x)
        x = self.flat(x)

        x = self.kan1(x) 
        x = F.log_softmax(x, dim=1)

        return x

In [22]:
convkan_types_map = {0: ConvKAN_1conv, 1: ConvKAN_2conv, 2: KKAN_Convolutional_Network}

### Create and train model

##### utility funcs

In [23]:
# see total number of params
def get_num_params(model):
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    pytorch_total_params_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

    return pytorch_total_params, pytorch_total_params_trainable

##### New util codes
copied from: https://github.com/AntonioTepsich/Convolutional-KANs/blob/master/experiment_28x28.ipynb

In [24]:
def train_batch_loader(model, device, train_loader, optimizer, epoch, criterion):
    """
    Train the model for one epoch

    Args:
        model: the neural network model
        device: cuda or cpu
        train_loader: DataLoader for training data
        optimizer: the optimizer to use (e.g. SGD)
        epoch: the current epoch
        criterion: the loss function (e.g. CrossEntropy)

    Returns:
        avg_loss: the average loss over the training set
    """

    model.to(device)
    model.train()
    train_loss = 0
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):
        # Recall that GPU is optimized for the operations we are dealing with
        data, target = data.to(device), target.to(device)
        # Reset the optimizer
        optimizer.zero_grad()
        # Push the data forward through the model layers
        output = model(data)
        # Get the loss
        loss = criterion(output, target)
        # Keep a running total
        train_loss += loss.item()
        # Backpropagate
        loss.backward()
        optimizer.step()
        
    # return average loss for the epoch
    avg_loss = train_loss / (batch_idx+1)
    # print('Training set: Average loss: {:.6f}'.format(avg_loss))
    return avg_loss

def test_batch_loader(model, device, test_loader, criterion):
    """
    Test the model

    Args:
        model: the neural network model
        device: cuda or cpu
        test_loader: DataLoader for test data
        criterion: the loss function (e.g. CrossEntropy)

    Returns:
        test_loss: the average loss over the test set
        accuracy: the accuracy of the model on the test set
        precision: the precision of the model on the test set
        recall: the recall of the model on the test set
        f1: the f1 score of the model on the test set
    """

    model.eval()
    test_loss = 0
    correct = 0
    all_targets = []
    all_predictions = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            # Get the predicted classes for this batch
            output = model(data)
            
            # Calculate the loss for this batch
            test_loss += criterion(output, target).item()
            
            # Calculate the accuracy for this batch
            _, predicted = torch.max(output.data, 1)
            correct += (target == predicted).sum().item()

            # Collect all targets and predictions for metric calculations
            all_targets.extend(target.view_as(predicted).cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    # Normalize test loss
    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)

    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%), Precision: {:.2f}, Recall: {:.2f}, F1 Score: {:.2f}\n'.format(
    #     test_loss, correct, len(test_loader.dataset), accuracy, precision, recall, f1))

    return test_loss, accuracy

In [25]:
def train_process(model, num_epochs, loss_fn, train_loader, test_loader, optimizer, scheduler=None):
    res = {'train_loss': [], 'test_loss': [], 'test_acc': []}
    for n in tqdm(range(num_epochs)):
        train_loss = train_batch_loader(model, 'cuda', train_loader, optimizer, n, loss_fn)
        # test_loss, test_acc, precision, recall, f1 = test_batch_loader(model, 'cuda', test_loader, loss_fn)
        test_loss, test_acc = test_batch_loader(model, 'cuda', test_loader, loss_fn)
        if scheduler:
            scheduler.step()
        
        res['train_loss'].append(train_loss)
        res['test_loss'].append(test_loss)
        res['test_acc'].append(test_acc)
    
    return res

##### Create different types of models and train them
We want to use the same optimization method, batch size, etc. to compare the different architectures fairly \
This might cause slower training

In [26]:
'''
create, train, then discard a model
our objective is to figure out the efficacy of different shaped models
'''
def create_and_train(model_type, width, epochs, lr, batch_size=-1):
    dset = dataset_flattened
    
    # create model
    if model_type == 'mlp':
        model = NeuralNetwork(width).to(device='cuda')
    elif model_type == 'kan':
        model = KAN(width, grid_size=3, spline_order=3).to(device='cuda')
    elif model_type == 'convkan':
        # TODO: change to use width
        model = convkan_types_map[width]().to(device='cuda')
        dset = dataset_channeled

    # record num of params
    num_params = get_num_params(model)
    
    # use categorical cross entropy loss
    loss_fn = torch.nn.CrossEntropyLoss()
    # learning rate and adam optimizer
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
    
    # train as any vanilla pytorch model
    start = time.time()
    # results = train(model, epochs, loss_fn, dset, optimizer, batch_size=batch_size)
    results = train_process(model, epochs, loss_fn, dset['train'], dset['test'], optimizer, scheduler=scheduler)
    end = time.time()

    # free the memory
    model.cpu()
    del model
    gc.collect()
    torch.cuda.empty_cache()

    return {'model_type': model_type, 'width': width, 'num_params': num_params, 'train_result': results, 'train_time': (end - start)}

In [27]:
kan_shapes = [
    [784, 10],
    [784, 16, 10],
    [784, 32, 10],
    [784, 64, 10],
    [784, 128, 10],
    [784, 32, 32, 10],
    [784, 64, 64, 10],
]

mlp_shapes = [
    [784, 10],
    [784, 16, 10],
    [784, 32, 10],
    [784, 64, 10],
    [784, 128, 10],
    [784, 256, 10],
    [784, 32, 32, 10],
    [784, 64, 64, 10],
    [784, 128, 128, 10],
    [784, 256, 256, 10],
    [784, 512, 512, 10],
]

convkan_types = convkan_types_map.keys()

In [28]:
results = []

In [29]:
epoch_ct = 40
# epoch_ct = 1

In [30]:
for shape in kan_shapes:
    print('Training KAN shape:', shape)
    res = create_and_train('kan', shape, epoch_ct, lr=1e-3)
    print('train time:', res['train_time'])
    print('------------------------------------------------------------')
    results.append(res)

for shape in convkan_types:
    print('Training ConvKAN type:', shape)
    res = create_and_train('convkan', shape, epoch_ct, lr=1e-3)
    print('train time:', res['train_time'])
    print('------------------------------------------------------------')
    results.append(res)

for shape in mlp_shapes:
    print('Training MLP shape:', shape)
    res = create_and_train('mlp', shape, epoch_ct, lr=1e-3)
    print('train time:', res['train_time'])
    print('------------------------------------------------------------')
    results.append(res)

Training KAN shape: [784, 10]


100%|███████████████████████████████████████████████████████████████████████████████| 40/40 [00:57<00:00,  1.44s/it]


train time: 57.71826672554016
------------------------------------------------------------
Training KAN shape: [784, 16, 10]


100%|███████████████████████████████████████████████████████████████████████████████| 40/40 [01:10<00:00,  1.77s/it]


train time: 70.62232851982117
------------------------------------------------------------
Training KAN shape: [784, 32, 10]


...

train time: 44.21124076843262
------------------------------------------------------------
Training MLP shape: [784, 256, 256, 10]


100%|███████████████████████████████████████████████████████████████████████████████| 40/40 [00:48<00:00,  1.20s/it]


train time: 48.14005398750305
------------------------------------------------------------
Training MLP shape: [784, 512, 512, 10]


100%|███████████████████████████████████████████████████████████████████████████████| 40/40 [00:44<00:00,  1.11s/it]

train time: 44.39455962181091
------------------------------------------------------------


In [31]:
results

[{'model_type': 'kan',
  'width': [784, 10],
  'num_params': (62720, 62720),
  'train_result': {'train_loss': [0.838990182318586,
    0.5200798833623845,
    0.4686173620376181,
    0.44641085064157526,
    0.4327016163379588,
    0.4240541048506473,
    0.4172017543873888,
    0.4124736603270186,
    0.4092126345380824,
    0.40685463344797174,
    0.405140664983303,
    0.40294047911116415,
    0.4017946201436063,
    0.4010421858188954,
    0.40010680537274546,
    0.39969822688305634,
    0.39878858936593886,
    0.3986085350209094,
    0.3982203347885862,
    0.3979123854256691,
    0.397643252002432,
    0.39770095766858854,
    0.3975338933315683,
    0.3971603459500252,
    0.39782198287071063,
    0.3971752736162632,
    0.39730572605386694,
    0.39709142839654965,
    0.39735567569732666,
    0.397542302532399,
    0.39716964059687676,
    0.39690654594847496,
    0.397364282861669,
    0.3967489796750089,
    0.3973621606826782,
    0.39680553268879015,
    0.39705689638218

In [32]:
# save results to file
f = open('test_results_fashionMNIST_2.pickle', 'wb')
pickle.dump(results, f)
f.close()

### Extras